# Import

In [1]:
import os
import random

import pandas as pd
import numpy as np

from PIL import Image
from tqdm import tqdm 

from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader, Subset
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch import nn, optim

from sklearn.metrics import log_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


# Hyperparameter Setting

In [2]:
CFG = {
    'IMG_SIZE': 224,
    'BATCH_SIZE': 32,
    'EPOCHS': 100,
    'LEARNING_RATE': 0.0002,
    'SEED' : 42
}

# Fixed RandomSeed

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

# CustomDataset

In [4]:
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None, is_test=False):
        self.root_dir = root_dir
        self.transform = transform
        self.is_test = is_test
        self.samples = []

        if is_test:
            # 테스트셋: 라벨 없이 이미지 경로만 저장
            for fname in sorted(os.listdir(root_dir)):
                if fname.lower().endswith(('.jpg')):
                    img_path = os.path.join(root_dir, fname)
                    self.samples.append((img_path,))
        else:
            # 학습셋: 클래스별 폴더 구조에서 라벨 추출
            self.classes = sorted(os.listdir(root_dir))
            self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}

            for cls_name in self.classes:
                cls_folder = os.path.join(root_dir, cls_name)
                for fname in os.listdir(cls_folder):
                    if fname.lower().endswith(('.jpg')):
                        img_path = os.path.join(cls_folder, fname)
                        label = self.class_to_idx[cls_name]
                        self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if self.is_test:
            img_path = self.samples[idx][0]
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image
        else:
            img_path, label = self.samples[idx]
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, label


# Data Load

In [5]:
train_root = './train'
test_root = './test'

In [6]:
# train_transform = transforms.Compose([
#     transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
#     transforms.Grayscale(num_output_channels=3),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5],
#                          std=[0.5, 0.5, 0.5])
# ])

# val_transform = transforms.Compose([
#     transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
#     transforms.Grayscale(num_output_channels=3),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.5, 0.5, 0.5],
#                          std=[0.5, 0.5, 0.5])
# ])


train_transform = transforms.Compose([
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((CFG['IMG_SIZE'], CFG['IMG_SIZE'])),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [7]:
# 전체 데이터셋 로드
full_dataset = CustomImageDataset(train_root, transform=None)
print(f"총 이미지 수: {len(full_dataset)}")

targets = [label for _, label in full_dataset.samples]
class_names = full_dataset.classes

# Stratified Split
train_idx, val_idx = train_test_split(
    range(len(targets)), test_size=0.2, stratify=targets, random_state=42
)

# Subset + transform 각각 적용
train_dataset = Subset(CustomImageDataset(train_root, transform=train_transform), train_idx)
val_dataset = Subset(CustomImageDataset(train_root, transform=val_transform), val_idx)
print(f'train 이미지 수: {len(train_dataset)}, valid 이미지 수: {len(val_dataset)}')


# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False)

총 이미지 수: 33137
train 이미지 수: 26509, valid 이미지 수: 6628


# Model Define

In [9]:
%pip install timm

   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 67.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

class ConvNeXt(nn.Module):
    def __init__(self, num_classes=396):
        super().__init__()

        # ConvNeXtV2 Base
        self.convnext = timm.create_model("convnextv2_base", pretrained=True)
        in_features1 = self.convnext.get_classifier().in_features
        self.convnext.reset_classifier(num_classes)  # fc 재설정

        # 모든 레이어 파인튜닝 (requires_grad=True)
        self._unfreeze_all(self.convnext)

    def forward(self, x):
        logits1 = self.convnext(x)

        probs1 = F.softmax(logits1, dim=1)
        
        return probs1

    def _unfreeze_all(self, model):
        for param in model.parameters():
            param.requires_grad = True


# Train / Validation

In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import log_loss
from tqdm import tqdm

# 모델 생성
model = ConvNeXt(num_classes=len(class_names)).to(device)

# 손실 함수 및 옵티마이저
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = optim.AdamW(model.parameters(), lr=CFG['LEARNING_RATE'])

# Early Stopping 설정
patience = 5
early_stop_counter = 0
best_logloss = float('inf')

for epoch in range(CFG['EPOCHS']):
    # ========== TRAIN ==========
    model.train()
    train_loss = 0.0

    for images, labels in tqdm(train_loader, desc=f"[Epoch {epoch+1}/{CFG['EPOCHS']}] Training"):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_loader)

    # ========== VALIDATION ==========
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    all_probs = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"[Epoch {epoch+1}/{CFG['EPOCHS']}] Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            # 정확도 계산
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            # LogLoss 계산용
            probs = F.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = 100 * correct / total
    val_logloss = log_loss(all_labels, all_probs, labels=list(range(len(class_names))))

    # 로그 출력
    print(f"[Epoch {epoch+1}] 🔹 Train Loss: {avg_train_loss:.4f} | 🔸 Valid Loss: {avg_val_loss:.4f} | ✅ Valid Acc: {val_accuracy:.2f}% | 📉 LogLoss: {val_logloss:.5f}")

    # ========== EARLY STOPPING ==========
    if val_logloss < best_logloss:
        best_logloss = val_logloss
        early_stop_counter = 0
        torch.save(model.state_dict(), f'convnext_1epoch.pth')
        print(f"📦 Best model saved at epoch {epoch+1} (logloss: {val_logloss:.5f})")
    else:
        early_stop_counter += 1
        print(f"⏸ No improvement for {early_stop_counter} epoch(s).")

        if early_stop_counter >= patience:
            print(f"🛑 Early stopping triggered at epoch {epoch+1}")
            break


[Epoch 1/100] Validation: 100%|██████████| 208/208 [01:36<00:00,  2.16it/s]
c:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


[Epoch 1] 🔹 Train Loss: 5.9815 | 🔸 Valid Loss: 5.9814 | ✅ Valid Acc: 0.24% | 📉 LogLoss: 5.98142
📦 Best model saved at epoch 1 (logloss: 5.98142)


[Epoch 2/100] Validation: 100%|██████████| 208/208 [01:30<00:00,  2.30it/s]
c:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


[Epoch 2] 🔹 Train Loss: 5.9815 | 🔸 Valid Loss: 5.9814 | ✅ Valid Acc: 0.26% | 📉 LogLoss: 5.98139
📦 Best model saved at epoch 2 (logloss: 5.98139)


[Epoch 3/100] Validation: 100%|██████████| 208/208 [01:29<00:00,  2.32it/s]
c:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


...

[Epoch 21/100] Validation: 100%|██████████| 208/208 [01:29<00:00,  2.33it/s]


[Epoch 21] 🔹 Train Loss: 5.9814 | 🔸 Valid Loss: 5.9813 | ✅ Valid Acc: 0.26% | 📉 LogLoss: 5.98134
⏸ No improvement for 4 epoch(s).


[Epoch 22/100] Validation: 100%|██████████| 208/208 [01:29<00:00,  2.32it/s]

[Epoch 22] 🔹 Train Loss: 5.9814 | 🔸 Valid Loss: 5.9813 | ✅ Valid Acc: 0.27% | 📉 LogLoss: 5.98133
⏸ No improvement for 5 epoch(s).
🛑 Early stopping triggered at epoch 22



c:\Users\user\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:2956: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


# Inference

# Submission